In [1]:
import torch
import numpy as np
from pathlib import Path
import argparse
from neuralop.models import UNO
from magplot.base import create_mesh, mag_plotter
import pyvista as pv
pv.start_xvfb()
pv.set_jupyter_backend('static')

import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from rtmag.test.eval_plot import plot_sample

from rtmag.test.eval import evaluate


from tqdm import tqdm
from torch.optim import Adam
import gc
from rtmag.train.diff_torch_batch import curl, divergence
from torchmetrics.regression import ConcordanceCorrCoef, MeanSquaredError

In [2]:
meta_path = Path("/home/usr/workspace/base/uno_pi_cc_hnorm_1_unit_aug_ccc_square")
checkpoint = torch.load(meta_path / "best_model.pt", map_location=device)

args = argparse.Namespace()
info = np.load(meta_path / 'args.npy', allow_pickle=True).item()
for key, value in info.items():
        args.__dict__[key] = value

# b_norm = args.data["b_norm"]

model = UNO(
        hidden_channels = args.model["hidden_channels"],
        in_channels = args.model["in_channels"],
        out_channels = args.model["out_channels"],
        lifting_channels = args.model["lifting_channels"],
        projection_channels = args.model["projection_channels"],
        n_layers = args.model["n_layers"],

        factorization = args.model["factorization"],
        implementation = args.model["implementation"],
        rank = args.model["rank"],

        uno_n_modes = args.model["uno_n_modes"], 
        uno_out_channels = args.model["uno_out_channels"],
        uno_scalings = args.model["uno_scalings"],
    ).to(device)

checkpoint = torch.load(meta_path / 'best_model.pt')

model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [3]:
idx = 239

data_path = Path('/mnt/f/isee_dataset/12673/input/').glob('*.npz')
data_path = sorted(data_path)

label_path = Path('/mnt/f/isee_dataset/12673/label/').glob('*.npz')
label_path = sorted(label_path)

print(data_path[idx].name)
print(label_path[idx].name)

input_12673_20170906_083600.npz
label_12673_20170906_083600.npz


In [4]:
inputs = np.load(data_path[idx])
model_input = torch.from_numpy(inputs['input'])[None, ...]

b_norm = torch.max(torch.abs(model_input)).item()
# [batch_size, 3, 513, 257, 1]
model_input = model_input[:, :, :-1, :-1, :] / b_norm # remove duplicated periodic boundary
model_input = model_input.to(device)
model_input = torch.permute(model_input, (0, 4, 3, 2, 1))

In [5]:
labels = model(model_input).detach()
labels.shape

torch.Size([1, 256, 256, 512, 3])

In [6]:
model = model.train()
optimizer = Adam(model.parameters(), lr=args.training['learning_late'])

with tqdm(range(200)) as tqdm_loader_train:
        for iteration in tqdm_loader_train:
                gc.collect()
                torch.cuda.empty_cache()

                loss = {}
                # [b, z, y, x, 3] 
                outputs = model(model_input).to(device)

                # [b, z, y, x, 3] -> [b, z, ...]
                opts = torch.flatten(outputs, start_dim=2)
                lbls = torch.flatten(labels, start_dim=2)

                # [b, z, ...] -> [b, ..., z]
                opts = torch.permute(opts, (0, 2, 1))
                lbls = torch.permute(lbls, (0, 2, 1))

                # mse loss
                mse = MeanSquaredError().to(device)
                loss_mse = torch.mean(mse(opts.flatten(), lbls.flatten()))
                loss['mse'] = loss_mse

                # ccc loss
                # ccc = ConcordanceCorrCoef(num_outputs=opts.shape[-1]).to(device)
                # loss_ccc = 0.0
                # if args.training.get('ccc_square', False):
                #         for i in range(opts.shape[0]):
                #                 loss_ccc += torch.mean(torch.square(1.0 - ccc(opts[i], lbls[i])))
                # else:
                #         for i in range(opts.shape[0]):
                #                 loss_ccc += torch.mean(torch.abs(1.0 - ccc(opts[i], lbls[i])))
                # loss_ccc /= opts.shape[0]
                # loss['ccc'] = loss_ccc
                
                # [b, z, y, x, 3] -> [b, x, y, z, 3]
                b = torch.permute(outputs, (0, 3, 2, 1, 4))
                B = torch.permute(labels, (0, 3, 2, 1, 4))


                # unnormalization
                if args.data["dataset_name"] == "Hnorm_Square_Unit_Aug":
                        divisor = (1 / np.arange(1, b.shape[2] + 1)**2 ).reshape(1, 1, -1, 1).astype(np.float32)
                else:
                        divisor = (1 / np.arange(1, b.shape[2] + 1)).reshape(1, 1, -1, 1).astype(np.float32)
                        
                divisor = torch.from_numpy(divisor).to(device)
                b = b * divisor
                B = B * divisor
                
                # boundary condition loss
                loss_bc = 0.0
                # bottom (z=0)
                loss_bc += torch.mean(torch.square(b[:, :, :, 0, :] - B[:, :, :, 0, :]))
                loss['bc'] = loss_bc

                dx = torch.from_numpy(np.array([1.0]).astype(np.float32)).reshape(-1, 1)[None, ...].to(device)
                dy = torch.from_numpy(np.array([1.0]).astype(np.float32)).reshape(-1, 1)[None, ...].to(device)
                dz = torch.from_numpy(np.array([1.0]).astype(np.float32)).reshape(-1, 1)[None, ...].to(device)

                # force-free loss
                bx, by, bz = b[..., 0], b[..., 1], b[..., 2]
                jx, jy, jz = curl(bx, by, bz, dx, dy, dz)
                b = torch.stack([bx, by, bz], -1)
                j = torch.stack([jx, jy, jz], -1)

                jxb = torch.cross(j, b, -1)
                loss_ff = (jxb**2).sum(-1) / ((b**2).sum(-1) + 1e-7)
                loss_ff = torch.mean(loss_ff)
                loss['ff'] = loss_ff

                # divergence-less loss
                div_b = divergence(bx, by, bz, dx, dy, dz)
                loss_div = torch.mean(torch.square(div_b))
                loss['div'] = loss_div


                los = 0.01*args.training['w_mse']*loss['mse'] \
                + args.training['w_bc']*loss['bc'] \
                + 100*args.training['w_ff']*loss['ff'] \
                + 100*args.training['w_div']*loss['div'] \
                
                print(f"iter:{iteration}, loss:{los.item():2g}, mse:{loss_mse.item():.2g}, bc:{loss_bc.item():.2g}, ff:{loss_ff.item():.2g}, div:{loss_div.item():.2g}")

                optimizer.zero_grad()
                los.backward()
                optimizer.step()

  0%|          | 0/200 [00:00<?, ?it/s]

iter:0, loss:0.00119064, mse:0, bc:0, ff:8.8e-06, div:3.1e-06


  0%|          | 1/200 [00:03<10:39,  3.21s/it]

iter:1, loss:0.00171149, mse:0.0052, bc:5e-05, ff:8e-06, div:3.6e-06


In [ ]:
model_output = model(model_input)
# [512, 256, 256, 3]
b = model_output.detach().cpu().numpy().transpose(0, 3, 2, 1, 4)[0]
divi = (b_norm / np.arange(1, b.shape[2] + 1)).reshape(1, 1, -1, 1)
b = b * divi

In [ ]:
print(b_norm)

In [ ]:
B = np.load(label_path[idx])["label"][:, :-1, :-1, :-1]
B = B.transpose(1, 2, 3, 0)
B.shape

In [ ]:
evaluate(b, B)

In [ ]:
vmin=-2500
vmax=2500
i_siz=b.shape[0] / 2
j_siz=b.shape[1] / 2
i_res=16
j_res=16
window_size=(1200, 800)
zoom=1.5
max_time=10000
camera_position = 'xy'
b_title = ''
title_fontsize = 10

bx = b[..., 0]
by = b[..., 1]
bz = b[..., 2]
mesh = create_mesh(bx, by, bz)
b_plot = mag_plotter(mesh)
b_tube, b_bottom, b_dargs = b_plot.create_mesh(i_siz=i_siz, j_siz=j_siz, i_resolution=i_res, j_resolution=j_res, vmin=vmin, vmax=vmax, max_time=max_time)


p = pv.Plotter(off_screen=False, window_size=window_size)
p.add_mesh(b_plot.grid.outline())
p.add_mesh(b_bottom, cmap='gray', **b_dargs)
p.add_mesh(b_tube, lighting=False, color='blue')
p.camera_position = camera_position
p.add_title(b_title, font_size=title_fontsize)
p.camera.zoom(zoom)
p.show()

In [ ]:
vmin=-2500
vmax=2500
i_siz=B.shape[0] / 2
j_siz=B.shape[1] / 2
i_res=16
j_res=16
window_size=(1200, 800)
zoom=1.5
max_time=10000
camera_position = 'xy'
b_title = ''
title_fontsize = 10

bx = B[..., 0]
by = B[..., 1]
bz = B[..., 2]
mesh = create_mesh(bx, by, bz)
b_plot = mag_plotter(mesh)
b_tube, b_bottom, b_dargs = b_plot.create_mesh(i_siz=i_siz, j_siz=j_siz, i_resolution=i_res, j_resolution=j_res, vmin=vmin, vmax=vmax, max_time=max_time)


p = pv.Plotter(off_screen=False, window_size=window_size)
p.add_mesh(b_plot.grid.outline())
p.add_mesh(b_bottom, cmap='gray', **b_dargs)
p.add_mesh(b_tube, lighting=False, color='blue')
p.camera_position = camera_position
p.add_title(b_title, font_size=title_fontsize)
p.camera.zoom(zoom)
p.show()

In [ ]:
plot_sample(b, B, v_mm=2500)